In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Load the datasets
cp_count_sanchez = pd.read_csv('CP_count_Sanchez.csv')

# Function to merge data with CP_count_Sanchez
def merge_data(data, cp_data):
    return data.merge(cp_data, on='INCHIKEY')


# Extract assays (columns) to be used for logistic regression
assay_columns = [col for col in cp_count_sanchez.columns if col not in ['INCHIKEY', 'Unnamed: 0','Cells_Number_Object_Number','Cells_Neighbors_FirstClosestObjectNumber_5',
 'Cells_Neighbors_FirstClosestObjectNumber_Adjacent',
 'Cells_Neighbors_SecondClosestObjectNumber_5',
 'Cells_Neighbors_SecondClosestObjectNumber_Adjacent',
 'Cells_Parent_Nuclei',
 'Cytoplasm_Number_Object_Number',
 'Cytoplasm_Parent_Cells',
 'Cytoplasm_Parent_Nuclei',
 'Nuclei_Neighbors_FirstClosestObjectNumber_1',
 'Nuclei_Neighbors_SecondClosestObjectNumber_1',
 'Nuclei_Number_Object_Number',
 'InChIKey']]

# Replace -1 with NaN only in assay columns
cp_count_sanchez[assay_columns] = cp_count_sanchez[assay_columns].replace(-1, np.nan)


train_cols = ['Cells_Number_Object_Number', 'Cells_Neighbors_FirstClosestObjectNumber_5', 'Cells_Neighbors_FirstClosestObjectNumber_Adjacent', 'Cells_Neighbors_SecondClosestObjectNumber_5', 'Cells_Neighbors_SecondClosestObjectNumber_Adjacent']

In [11]:
cp_count_sanchez

,Unnamed: 0,INCHIKEY,1,2,3,4,5,6,7,8,...,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number,InChIKey
0,0,AACRWZVDRSTLKY-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.054688,-0.054688,0.859375,0.859375,0.859375,0.859375,1.445312,1.007812,0.859375,AACRWZVDRSTLKY-UHFFFAOYSA-N
1,1,AACUKVXTFOXDGE-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.171875,-1.171875,-1.148438,-1.148438,-1.148438,-1.148438,-1.101562,-1.453125,-1.148438,AACUKVXTFOXDGE-UHFFFAOYSA-N
2,2,AADCDMQTJNYOSS-LBPRGKRZSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.425781,-0.425781,0.453125,0.453125,0.453125,0.453125,-0.117188,-0.531250,0.453125,AADCDMQTJNYOSS-LBPRGKRZSA-N
3,3,AADORYZVGJDNSZ-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.123188,1.123188,0.521739,0.521739,0.521739,0.521739,1.536232,0.601449,0.521739,AADORYZVGJDNSZ-UHFFFAOYSA-N
4,4,AAEVYOVXGOFMJO-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.882812,-6.882812,-7.218750,-7.218750,-7.218750,-7.218750,-7.679688,-7.773438,-7.218750,AAEVYOVXGOFMJO-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567,10567,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.609375,0.609375,0.679688,0.679688,0.679688,0.679688,0.757812,0.757812,0.679688,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N
10568,10568,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.242188,-10.242188,-10.746094,-10.746094,-10.746094,-10.746094,-11.167969,-10.281250,-10.746094,ZZUFCTLCJUWOSV-UHFFFAOYSA-N
10569,10569,ZZUZYEMRHCMVTB-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-16.171875,-16.171875,-16.304688,-16.304688,-16.304688,-16.304688,-16.335938,-15.152344,-16.304688,ZZUZYEMRHCMVTB-UHFFFAOYSA-N
10570,10570,ZZVUWRFHKOJYTH-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.640625,-0.640625,-1.664062,-1.664062,-1.664062,-1.664062,-1.257812,-0.289062,-1.664062,ZZVUWRFHKOJYTH-UHFFFAOYSA-N


In [12]:
cp_count_sanchez.columns[-13:]

Index(['Cells_Number_Object_Number',
       'Cells_Neighbors_FirstClosestObjectNumber_5',
       'Cells_Neighbors_FirstClosestObjectNumber_Adjacent',
       'Cells_Neighbors_SecondClosestObjectNumber_5',
       'Cells_Neighbors_SecondClosestObjectNumber_Adjacent',
       'Cells_Parent_Nuclei', 'Cytoplasm_Number_Object_Number',
       'Cytoplasm_Parent_Cells', 'Cytoplasm_Parent_Nuclei',
       'Nuclei_Neighbors_FirstClosestObjectNumber_1',
       'Nuclei_Neighbors_SecondClosestObjectNumber_1',
       'Nuclei_Number_Object_Number', 'InChIKey'],
      dtype='object')

In [13]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# Function to perform logistic regression and get the best AUC-ROC score
def perform_logistic_regression(train, val, test, assay):
    X_train = train[train_cols]
    y_train = train[assay]
    X_val = val[train_cols]
    y_val = val[assay]
    X_test = test[train_cols]
    y_test = test[assay]
    
    # Check if any of the datasets are empty or only one class is present
    if (len(y_train) == 0 or len(y_val) == 0 or len(y_test) == 0 or
        len(y_train.unique()) == 1 or len(y_val.unique()) == 1 or len(y_test.unique()) == 1):
        return None, None, None

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    # Convert back to DataFrame
    X_train = pd.DataFrame(X_train, columns=train_cols)
    X_val = pd.DataFrame(X_val, columns=train_cols)
    X_test = pd.DataFrame(X_test, columns=train_cols)
    
    C_param_range = [10**i for i in range(-6, 7)]

    best_auc = 0
    best_C = None
    best_model = LogisticRegression(C=1, max_iter=3000, random_state=42)
    best_model.fit(X_train, y_train)

    for C in C_param_range:
        model = LogisticRegression(C=C, max_iter=3000, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluate on validation set
        y_prob = model.predict_proba(X_val)[:, 1]
        val_auc = roc_auc_score(y_val, y_prob)
        
        if val_auc > best_auc:
            best_auc = val_auc
            best_C = C
            best_model = model

    # Retrain the best model on the combined training and validation set
    X_train_val = pd.concat([X_train, X_val])
    y_train_val = pd.concat([y_train, y_val])
    best_model.fit(X_train_val, y_train_val)

    # Evaluate the best model on the test set
    y_prob = best_model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_prob)

    return best_C, best_auc, test_auc

def process_split(train_path, val_path, test_path, split_label):
    train = pd.read_csv(train_path)
    val = pd.read_csv(val_path)
    test = pd.read_csv(test_path)

    # Merge the datasets
    train_merged = merge_data(train, cp_count_sanchez)
    val_merged = merge_data(val, cp_count_sanchez)
    test_merged = merge_data(test, cp_count_sanchez)

    # Perform logistic regression for each assay and store the results
    results = []
    for assay in tqdm(assay_columns, desc=f"Processing {split_label}"):
        train_assay = train_merged.dropna(subset=[assay])[[assay] + train_cols]
        val_assay = val_merged.dropna(subset=[assay])[[assay] + train_cols]
        test_assay = test_merged.dropna(subset=[assay])[[assay] + train_cols]
        
        best_C, val_auc, test_auc = perform_logistic_regression(train_assay, val_assay, test_assay, assay)
        if best_C is not None:
            results.append({'assay': assay, 'best_C': best_C, 'val_auc': val_auc, 'test_auc': test_auc, 'split': split_label})

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Paths to data splits
data_splits = [
    ('data/datasplit1-train.csv', 'data/datasplit1-val.csv', 'data/datasplit1-test.csv', 'split1'),
    ('data/datasplit2-train.csv', 'data/datasplit2-val.csv', 'data/datasplit2-test.csv', 'split2'),
    ('data/datasplit3-train.csv', 'data/datasplit3-val.csv', 'data/datasplit3-test.csv', 'split3')
]

# Process each split and combine results
all_results = pd.DataFrame()
for train_path, val_path, test_path, split_label in data_splits:
    split_results = process_split(train_path, val_path, test_path, split_label)
    all_results = pd.concat([all_results, split_results], ignore_index=True)

# Save the combined results to a CSV file
all_results.to_csv('logistic_regression_results.csv', index=False)

Processing split3: 100%|██████████| 209/209 [00:50<00:00,  4.10it/s]


In [14]:
all_results

,assay,best_C,val_auc,test_auc,split
0,1,0.000001,0.666667,0.500000,split1
1,2,0.100000,0.933333,0.537037,split1
2,4,1000.000000,0.638889,0.428571,split1
3,5,1000.000000,1.000000,0.555556,split1
4,6,10000.000000,0.300000,0.689076,split1
...,...,...,...,...,...
530,203,0.000001,0.408213,0.605011,split3
531,204,0.000001,0.541667,0.538462,split3
532,205,0.000001,0.735294,0.689394,split3
533,208,0.000001,0.400000,0.384615,split3
